In [9]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss


In [32]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
img_height = 150
img_width = 150

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [33]:
def wrapper_generator(generator):
    for batch_x, batch_y in generator:
        yield [batch_x, batch_y], batch_y

In [34]:
train_gen_wrapper = wrapper_generator(train_generator)
validation_gen_wrapper = wrapper_generator(validation_generator)
test_gen_wrapper = wrapper_generator(test_generator)

In [35]:
train_gen_wrapper

<generator object wrapper_generator at 0x35d37b3e0>

In [36]:
class ArcFaceLossLayer(tf.keras.layers.Layer):
    def __init__(self, n_classes=2, s=30.0, m=0.50, **kwargs):
        super(ArcFaceLossLayer, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s  # Scale parameter
        self.m = m  # Margin parameter
        self.cos_m = tf.cos(m)
        self.sin_m = tf.sin(m)
        self.th = tf.cos(tf.constant(np.pi) - m)
        self.mm = tf.sin(tf.constant(np.pi) - m) * m

    def build(self, input_shape):
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 dtype='float32')

    def call(self, inputs, labels):
        # Normalize feature vectors and weights
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        # Dot product
        logits = x @ W
        # Add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + 1e-7, 1.0 - 1e-7))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - labels) + target_logits * labels
        # Rescale logits
        return logits * self.s


In [37]:
def build_model_with_arcface_loss(input_shape=(150, 150, 3), n_classes=2):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    # No activation on the last dense layer
    x = Dense(128)(x)  # This is the embedding layer
    # Note: ArcFace layer is typically applied during training only

    # Placeholder for labels
    labels = Input(shape=(n_classes,))
    # Instantiate ArcFace loss layer
    arcface_loss = ArcFaceLossLayer(n_classes=n_classes)(x, labels)

    model = Model(inputs=[inputs, labels], outputs=arcface_loss)
    return model


In [38]:
model = build_model_with_arcface_loss(input_shape=(150, 150, 3), n_classes=2)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=[tf.nn.softmax_cross_entropy_with_logits],
              metrics=['accuracy'])

# When fitting the model, ensure to pass both the images and labels as input,
# and similarly for validation. You might need to adjust your data generator
# or data loading mechanism to achieve this.


In [39]:
model.fit(
    train_gen_wrapper,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_gen_wrapper,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
43/43 [==============================] - 11s 254ms/step - loss: 6.4804 - accuracy: 0.5211 - val_loss: 4.6905 - val_accuracy: 0.7396
Epoch 2/20
43/43 [==============================] - 11s 263ms/step - loss: 5.1538 - accuracy: 0.5187 - val_loss: 3.9199 - val_accuracy: 0.7201
Epoch 3/20
43/43 [==============================] - 11s 266ms/step - loss: 3.7971 - accuracy: 0.5823 - val_loss: 3.4966 - val_accuracy: 0.7388
Epoch 4/20
43/43 [==============================] - 12s 288ms/step - loss: 2.6691 - accuracy: 0.6811 - val_loss: 2.2151 - val_accuracy: 0.7799
Epoch 5/20
43/43 [==============================] - 12s 283ms/step - loss: 1.9128 - accuracy: 0.7601 - val_loss: 1.4451 - val_accuracy: 0.8134
Epoch 6/20
43/43 [==============================] - 12s 279ms/step - loss: 1.3222 - accuracy: 0.8098 - val_loss: 1.5842 - val_accuracy: 0.8321
Epoch 7/20
43/43 [==============================] - 13s 297ms/step - loss: 1.1273 - accuracy: 0.8332 - val_loss: 1.1891 - val_accuracy: 0.8918

In [41]:
test_loss, test_accuracy = model.evaluate(test_gen_wrapper, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 70ms/step - loss: 1.6139 - accuracy: 0.9028
Test Loss: 1.6139473915100098, Test Accuracy: 0.9027777910232544
